In [16]:
# New technology used
#  %%capture
# SentencePiece as a supplement to NLTK

In [17]:
# Create an application that will allow users to input symptoms, which can then be paired with possible diseases and offer treatements
# Will use gradio to build this app

%%capture

!pip install gradio
!pip intall nltk
!pip install transformers
!pip install torch
!pip install sentencepiece
!pip install tensorflow
!pip install tensorflow_hub
!pip install tensorflow_text


In [18]:
from google.colab import files
uploaded = files.upload()

Saving symbipredict.csv to symbipredict (1).csv


In [19]:
# Imports needed for this application
import gradio as gr
import nltk
import transformers
import torch
import sentencepiece
import tensorflow
import tensorflow_hub
import tensorflow_text
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In this section we will pull in patient images if available.  Not sure how we can do this, so this is just a place holder for now.

In [20]:
# Pull in patient images(s) if available

pull_patient_images = True

In this section, we will use NLTK functions to establish our patient input.  Then we establish our Gradio interface.

In [26]:
# Download required NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

# Download the averaged_perceptron_tagger_eng resource
nltk.download('averaged_perceptron_tagger_eng')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


def extract_keywords(patient_feedback):
    """Extracts keywords from patient feedback using NLTK."""

    # Tokenize the feedback text
    tokens = word_tokenize(patient_feedback.lower())

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [w for w in tokens if not w.lower() in stop_words and w.isalnum()]

# Use Parts of Speech (POS) functions below to further refine the nltk process

    # Perform POS tagging
    pos_tags = nltk.pos_tag(filtered_tokens)

    # Extract nouns and adjectives as keywords
    keywords = [word for word, pos in pos_tags if pos.startswith('NN') or pos.startswith('JJ')]

    return keywords

# Now useNamed Entity Recognition (NER) functions below to ruther refine medical term extractions
import spacy

# Load spaCy model
nlp = spacy.load('en_core_web_sm')

def extract_named_entities(patient_feedback):
    """Extracts named entities from patient feedback using spaCy."""

    # Process the feedback text
    doc = nlp(patient_feedback)

    # Extract named entities
    named_entities = [(ent.text, ent.label_) for ent in doc.ents]

    return named_entities
# Load the disease databse from CSV here

disease_data = pd.read_csv('symbipredict.csv')

def suggest_diagnosis(patient_feedback):
    """Suggests possible diagnoses based on extracted keywords."""

    # Extract keywords from patient feedback
    keywords = extract_keywords(patient_feedback)
    print("Extracted Keywords:", keywords)

    possible_diagnoses = []

 # Iterate through the disease database
    for index, row in disease_data.iterrows():
        # Replace 'Disease' with the actual column name in your CSV containing the disease names
        disease_name = row['Disease']  # Assuming a 'Disease' column in your CSV

        # Get the correct column name for symptoms from your CSV file
        # Replace 'Symptom_Column_Name' with the actual column name
        symptom_column_name = 'Symptom_Column_Name'

   # Check if the symptom column exists before accessing it
        if symptom_column_name in disease_data.columns:
            disease_keywords = row[symptom_column_name].lower().split(',')
        else:
            print(f"Warning: Column '{symptom_column_name}' not found in disease data.")
            continue  # Skip to the next row if the column is not found


        # Check if any extracted keywords match the disease keywords
        if any(keyword in disease_keywords for keyword in keywords):
            possible_diagnoses.append(disease_name)

    if not possible_diagnoses:
        possible_diagnoses = ["Unable to determine a diagnosis based on the provided information."]


    return possible_diagnoses

# Example usage
feedback = "Patient reports severe headache and dizziness."
diagnosis = suggest_diagnosis(feedback)
print("Suggested Diagnosis:", diagnosis)


# Gradio Interface
iface = gr.Interface(
    fn=suggest_diagnosis,
    inputs=gr.Textbox(lines=5, placeholder="Describe your symptoms..."),
    outputs="text",
    title="Symptom Checker",
    description="Enter your symptoms, and we'll suggest possible diagnoses.",
)

iface.launch()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Extracted Keywords: ['patient', 'reports', 'severe', 'headache', 'dizziness']
Suggested Diagnosis: ['Unable to determine a diagnosis based on the provided information.']
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://026c93f369aec3208e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In the section below, we will use the patient's diagnosis to suggest possible treatments.
First, we have to load the treatment database that we have selected (name it here).  Then, we will run a process to link the disease to a Treatment  using Pandas Dataframe (for CSV)

In [27]:
# View patient responses
patient_symptoms
for symptom in patient_symptoms:
  print(symptom)

NameError: name 'patient_symptoms' is not defined

In [ ]:
#  Load and Access Treatment Database
treatment_database = pd.read_csv('treatment_database.csv')
treatment_database.head()

In [ ]:
# File import
import sqlite3
        conn = sqlite3.connect('treatment_database.db')
        cursor = conn.cursor()
        cursor.

In [ ]:
# Link the disease to a possible treatment
def get_treatments(diagnosed_disease):
        treatments = treatment_data[treatment_data['disease'] == diagnosed_disease]['treatment'].tolist()
        if treatments:
            return treatments
        else:
            return "No treatments found for this disease."